In [1]:
!pip install pandas
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [24]:
import pandas as pd
import numpy as np

# подгружаем датасет
data = pd.read_csv('StudentsPerformance.csv')

# удаляем строки с NA
data = data.dropna()

# преобразуем значения столбцов, которые имеют только два разных значения в бинарное представление
data['gender'] = np.where(data['gender'] == "female", 1, 0) # female - 1, male - 1
data['test_preparation_course'] = np.where(data['test_preparation_course'] == "completed", 1, 0) # completed - 1, none - 0
data['lunch'] = np.where(data['lunch'] == "standard", 1, 0)

In [25]:
# отделим значение 'test preparation course' от остальных данных
test_preparation_course = data.loc[:, data.columns.isin(['test_preparation_course'])]
data = data.loc[:, data.columns.isin(['gender', 'race', 'parental_level_of_education', 'lunch', 'math_score', 'reading_score', 'writing_score'])]

In [26]:
# переведем категориальные переменные в one hot encoding
race = pd.get_dummies(data.race, prefix='race')
educ = pd.get_dummies(data.parental_level_of_education, prefix='educ')

# удалим их из основых данных
data = data.drop(columns = ['race', 'parental_level_of_education'])

# добавим ohe датафреймы к основным данным
frames = [data, race, educ]
data = pd.concat(frames, axis = 1)

# удалим по одному столбцу из каждой eho переменной
data = data.drop(columns = ['race_group E', 'educ_some high school'])

In [27]:
data.head()

,gender,lunch,math_score,reading_score,writing_score,race_group A,race_group B,race_group C,race_group D,educ_associate's degree,educ_bachelor's degree,educ_high school,educ_master's degree,educ_some college
0,1,1,72,72,74,0,1,0,0,0,1,0,0,0
1,1,1,69,90,88,0,0,1,0,0,0,0,0,1
2,1,1,90,95,93,0,1,0,0,0,0,0,1,0
3,0,0,47,57,44,1,0,0,0,1,0,0,0,0
4,0,1,76,78,75,0,0,1,0,0,0,0,0,1


In [28]:
# разделим датасет на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, test_preparation_course, test_size = 0.33, random_state=1)

In [29]:
# построим классификатор типа решающее дерево (decision tree classifier)
from sklearn import tree

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(x_train, y_train)

In [30]:
# оценим точность классификатора с помощью метрик precision, recall и F1 на тестовой выборке
from sklearn.model_selection import cross_val_score

print("f1:", str(np.average(cross_val_score(dtc, x_test, y_test, scoring='f1'))))
print("precision:", str(np.average(cross_val_score(dtc, x_test, y_test, scoring='precision'))))
print("recall:", str(np.average(cross_val_score(dtc, x_test, y_test, scoring='recall'))))

f1: 0.5072194970035502
precision: 0.4680948616600791
recall: 0.46268115942028987


In [31]:
# построим классификатор типа случайный лес (random forest classifier)
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
# используем методы .values.ravel(), чтобы привести y_train к нужному формату
rfc = rfc.fit(x_train, y_train.values.ravel())

In [32]:
# оценим точность классификатора с помощью метрик precision, recall и F1 на тестовой выборке

# используем методы .values.ravel(), чтобы привести y_test к нужному формату
print("f1:", str(np.average(cross_val_score(rfc, x_test, y_test.values.ravel(), scoring='f1'))))
print("precision:", str(np.average(cross_val_score(rfc, x_test, y_test.values.ravel(), scoring='precision'))))
print("recall:", str(np.average(cross_val_score(rfc, x_test, y_test.values.ravel(), scoring='recall'))))

f1: 0.39473269278147327
precision: 0.49316849816849817
recall: 0.35289855072463766
